In [2]:
import pandas_market_calendars as mcal
import yfinance as yf
import pandas as pd
import numpy as np
import scipy.stats as stats

In [3]:
# Create a calendar
nse = mcal.get_calendar('NSE')

end_date = '2019-10-01'  # Starting date

# Show number of valid business days. Calculated by trial and error considering approximately 252 trading days in 1 calendar year.
nse.valid_days(start_date='2017-09-15', end_date=end_date).size

504

In [4]:
def get_historical_data(ticker, end_date, start_date):
    """
    Fetch historical stock data for a given ticker from Yahoo Finance.

    Parameters:
    - ticker: Stock ticker symbol (str).
    - end_date: The date from which the number of days are counted back(format: 'YYYY-MM-DD').
    - start_days: Beginning date of the period

    Returns:
    - DataFrame containing the historical data for the specified period.
    """
    
    # Download the data from Yahoo Finance
    data = yf.download(ticker, start=start_date, end=end_date)
    
    price_data = data['Adj Close']
    price_data.columns = ticker
    
    return price_data

In [14]:
portfolio_tickers=["DLF.NS","NTPC.NS","HDFCBANK.NS"]
historical_data =get_historical_data(portfolio_tickers,'2019-10-01','2017-09-17')
historical_data=np.log(historical_data/historical_data.shift(1)).dropna()

[*********************100%%**********************]  3 of 3 completed


In [6]:
historical_data.head()

,DLF.NS,NTPC.NS,HDFCBANK.NS
Date,,,
2017-09-19,-0.006012,-0.005795,0.005935
2017-09-20,-0.004994,-0.000460,-0.007126
2017-09-21,-0.025081,-0.005288,0.002381
2017-09-22,-0.064449,-0.008190,-0.013166
2017-09-25,-0.040582,-0.013439,0.001505


In [7]:
historical_data.shape

(499, 3)

In [8]:
def portfolio_vol(weights, covmat):
    """
    Computes the vol of a portfolio from a covariance matrix and constituent weights
    weights are a numpy array or N x 1 maxtrix and covmat is an N x N matrix
    """
    return (weights.T @ covmat @ weights)**0.5

def var_historic(r, level=5):
    """
    Returns the historic Value at Risk at a specified level
    i.e. returns the number such that "level" percent of the returns
    fall below that number, and the (100-level) percent are above
    """
    if isinstance(r, pd.DataFrame):
        return r.aggregate(var_historic, level=level)
    elif isinstance(r, pd.Series):
        return -np.percentile(r, 100-level)
    else:
        raise TypeError("Expected r to be a Series or DataFrame")

In [9]:
weights=np.array([0.4,0.4,0.2]) ## Defining portfolio weights
weighted_returns = historical_data.mul(weights, axis=1)
portfolio_returns = weighted_returns.sum(axis=1) ## Calculating portfolio returns
portfolio_returns

Date
2017-09-19   -0.003536
2017-09-20   -0.003607
2017-09-21   -0.011671
2017-09-22   -0.031688
2017-09-25   -0.021307
                ...   
2019-09-24   -0.003731
2019-09-25   -0.030002
2019-09-26    0.015423
2019-09-27   -0.007095
2019-09-30   -0.011662
Length: 499, dtype: float64

# Historic VaR

In [10]:
## Defining Configence Level in a 100 % scale
confidence_level=99 

## Calculating Historic VaR in percentage
var_h=var_historic(portfolio_returns, level=confidence_level) 

print(var_h*100,"%")

3.700210633227667 %


# Parametric VaR

In [11]:
covmat=historical_data.cov()
covmat

,DLF.NS,NTPC.NS,HDFCBANK.NS
DLF.NS,0.000748,0.000087,0.000084
NTPC.NS,0.000087,0.000125,0.000023
HDFCBANK.NS,0.000084,0.000023,0.000195


In [12]:
mean_return = portfolio_returns.mean()
print(mean_return*100,"%")

0.00439236243331152 %


In [13]:
covmat=historical_data.cov()
pvol=portfolio_vol(weights, covmat)

# Calculate the mean and standard deviation of returns
mean_return = portfolio_returns.mean()
std_return = pvol

# Calculate the z-score for the given confidence level
z_score = stats.norm.ppf(1-confidence_level/100)

# Calculate the Parametric VaR
parametric_var = -(mean_return + z_score * std_return)

parametric_var*100
print(parametric_var*100,"%")

3.222651634708013 %
